In [8]:
import os

import ipywidgets as widgets
from ipywidgets import interact

import numpy as np
import pandas as pd
from IPython import embed

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
if not os.path.exists(REPOS_ROOT := os.environ.get("REPOS_ROOT", "/mnt/rodrigo/DGX")):
    raise FileNotFoundError(f"{REPOS_ROOT} does not exist.")

In [18]:
import os, sys; os.chdir(REPOS_ROOT)
from paths import *
# from CardiacCOMA.config.load_config import load_yaml_config, to_dict
from CardiacMotionRL.utils.run_helpers import *
from CardiacMotionGWAS.helpers import EnsembleResults, loci_mapping, rec_ratio_per_run

2024-10-28 19:25:54 - INFO - Reading runs from cached file /mnt/rodrigo/DGX/CardiacMotionGWAS/results/run_info.csv.
2024-10-28 19:26:42 - INFO - Found 226 runs. Cached file will be overwritten.


___

Collect the region-wise summaries

In [19]:
# GWAS_RESULTS_DIR = "/mnt/data/workshop/workshop-user1/output/GWAS/cardiac_indices_ed_and_es/"
GWAS_RESULTS_DIR = "/home/rodrigo/01_repos/GWAS_pipeline/output/All_partitions_spatiotemporal_afterNov2023/" #cardiac_indices_ed_and_es/"
results = EnsembleResults(GWAS_RESULTS_DIR) #, top_n_runs_per_chamber=20)

2024-10-28 19:28:41 - INFO - File /home/rodrigo/01_repos/GWAS_pipeline/output/All_partitions_spatiotemporal_afterNov2023//region_assocs_df.pkl does not exist. Results will be cached.
2024-10-28 19:28:41 - INFO - Found 616 files under the folder /home/rodrigo/01_repos/GWAS_pipeline/output/All_partitions_spatiotemporal_afterNov2023//summaries
100%|██████████| 616/616 [00:03<00:00, 158.23it/s]
2024-10-28 19:28:45 - INFO - Collected GWAS summary data for 616 phenotypes.
2024-10-28 19:28:45 - INFO - Concatenating...
2024-10-28 19:28:46 - INFO - Assigning dynamic/static label to phenotypes...
2024-10-28 19:29:03 - INFO - Caching results to /home/rodrigo/01_repos/GWAS_pipeline/output/All_partitions_spatiotemporal_afterNov2023//region_assocs_df.pkl


In [20]:
results.region_assocs_df

,pheno,region,CHR,SNP,BP,AF,a_0,a_1,INFO,BETA,SE,T,P,run,full_pheno,chamber,variable_type,msd_static,msd_dynamic,rec_ratio
0,z000,chr1_1,1,rs28669173,1612540.0,0.089077,C,T,1.00000,0.039881,0.011793,3.3817,0.000721,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
1,z000,chr1_10,1,rs558403893,13212155.0,0.030030,A,G,0.50056,-0.099257,0.027040,-3.6707,0.000242,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
2,z000,chr1_100,1,1:196946422_GA_G,196946422.0,0.241000,GA,G,0.87623,0.025229,0.008287,3.0443,0.002333,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
3,z000,chr1_101,1,1:199116792_GTAGAGAATACTAGTACTACTATTCTCTAC_G,199116792.0,0.052985,GTAGAGAATACTAGTACTACTATTCTCTAC,G,0.81040,0.054066,0.016173,3.3430,0.000829,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
4,z000,chr1_102,1,rs12061209,199965257.0,0.003171,C,T,0.94644,-0.317740,0.087854,-3.6167,0.000299,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035491,z031,chr9_71,9,rs7863156,137516100.0,0.077410,T,G,0.93413,0.047351,0.012790,3.7023,0.000214,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119
1035492,z031,chr9_72,9,rs117449706,138058133.0,0.029018,G,A,0.97580,0.071982,0.019547,3.6825,0.000231,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119
1035493,z031,chr9_73,9,rs77922980,139263794.0,0.023379,G,A,0.98996,-0.071112,0.021688,-3.2789,0.001043,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119
1035494,z031,chr9_8,9,rs10976278,7395669.0,0.171210,C,T,0.98794,-0.033661,0.008741,-3.8511,0.000118,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119


In [21]:
results.region_assocs_df.query("P < 5e-8") #  & (results.region_assocs_df.pheno == "RVEF")]

,pheno,region,CHR,SNP,BP,AF,a_0,a_1,INFO,BETA,SE,T,P,run,full_pheno,chamber,variable_type,msd_static,msd_dynamic,rec_ratio
227,z000,chr11_2,11,rs2106230,3041248.0,0.026699,G,A,0.95545,0.113900,0.020766,5.4848,4.161022e-08,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
1393,z000,chr6_78,6,rs7453860,118565620.0,0.134310,T,C,0.98911,-0.061752,0.009660,-6.3928,1.644372e-10,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
1394,z000,chr6_79,6,rs11756438,118993632.0,0.468620,C,A,0.98930,0.039991,0.006615,6.0455,1.500721e-09,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
2591,z000,chr21_10,21,rs66921363,28866669.0,0.173910,A,G,0.98911,-0.049573,0.008727,-5.6803,1.352384e-08,0fe28cdc20ec4ef28b736e330b8fd2e6,0fe28cdc20ec4ef28b736e330b8fd2e6_z000,LV,static,0.205662,0.576910,0.251073
3074,z000,chr6_78,6,rs72967533,118655020.0,0.477410,T,C,0.99207,-0.057576,0.006607,-8.7143,3.019952e-18,0fe28cdc20ec4ef28b736e330b8fd2e6,0fe28cdc20ec4ef28b736e330b8fd2e6_z000,LV,static,0.205662,0.576910,0.251073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031847,z031,chr6_79,6,rs11756440,118993642.0,0.468470,C,A,0.98894,-0.042073,0.006608,-6.3666,1.950743e-10,c9cb87277db74c9d8d512560cb758902,c9cb87277db74c9d8d512560cb758902_z031,LV,dynamic,0.112960,0.448596,0.192499
1032337,z031,chr10_74,10,rs72840788,121415685.0,0.214560,G,A,0.97945,0.045015,0.008024,5.6103,2.030954e-08,d4a372ea29d8492592fae2971d274753,d4a372ea29d8492592fae2971d274753_z031,LV,dynamic,0.359540,0.781835,0.343003
1032441,z031,chr12_17,12,rs17287293,24770878.0,0.149700,A,G,1.00000,0.050636,0.009202,5.5026,3.760971e-08,d4a372ea29d8492592fae2971d274753,d4a372ea29d8492592fae2971d274753_z031,LV,dynamic,0.359540,0.781835,0.343003
1034332,z031,chr15_29,15,rs77310346,63814142.0,0.041617,C,T,0.99755,-0.095074,0.016402,-5.7965,6.817105e-09,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119


In [22]:
# Counts per chamber and variable type
counts_per_chamber = results.counts_per_chamber()

___

In [23]:
FACTOR = 2
MOSTLY_DYNAMIC = ( counts_per_chamber["dynamic"].sum(axis=1) > counts_per_chamber["static"].sum(axis=1)*FACTOR )
MOSTLY_STATIC = ( counts_per_chamber["static"].sum(axis=1) > counts_per_chamber["dynamic"].sum(axis=1)*FACTOR)
BOTH_DYN_AND_STAT = ~(counts_per_chamber["static"].sum(axis=1) > counts_per_chamber["dynamic"].sum(axis=1)*FACTOR) & ~(counts_per_chamber["dynamic"].sum(axis=1) > counts_per_chamber["static"].sum(axis=1)*FACTOR )

counts_per_chamber.loc[MOSTLY_DYNAMIC].to_csv("CardiacMotionGWAS/results/counts_mostly_dynamic.csv")
counts_per_chamber.loc[MOSTLY_STATIC].to_csv("CardiacMotionGWAS/results/counts_mostly_static.csv")
counts_per_chamber.loc[BOTH_DYN_AND_STAT].to_csv("CardiacMotionGWAS/results/counts_both_static_and_dynamic.csv")

In [24]:
counts_per_chamber.loc[MOSTLY_DYNAMIC]

variable_type dynamic          static         
chamber            LV RV LA RA     LV RV LA RA
chr3_34             2  1  0  0      1  0  0  0
KDM2A               0  3  0  0      0  1  0  0
GJA1                2  1  0  0      1  0  0  0
SYT10               4  1  0  0      2  0  0  0
BAG3                7  2  0  0      3  1  0  0

___

In [25]:
# results.loci_summary(only_static=True).reset_index().groupby("region").min("min_P").sort_values("min_P").head(60)
# results.loci_summary(only_dynamic=True).reset_index().groupby("region").min("min_P").sort_values("min_P").head(60)

In [26]:
results.region_assocs_df.query("P < 1e-7")

,pheno,region,CHR,SNP,BP,AF,a_0,a_1,INFO,BETA,SE,T,P,run,full_pheno,chamber,variable_type,msd_static,msd_dynamic,rec_ratio
227,z000,chr11_2,11,rs2106230,3041248.0,0.026699,G,A,0.95545,0.113900,0.020766,5.4848,4.161022e-08,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
421,z000,chr13_47,13,rs9301907,94426655.0,0.259470,A,G,0.99051,0.040230,0.007499,5.3651,8.132049e-08,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
1202,z000,chr5_103,5,rs29783,172606818.0,0.367380,C,T,1.00000,-0.036518,0.006819,-5.3552,8.586180e-08,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
1204,z000,chr5_105,5,rs59092768,175235880.0,0.087870,C,T,1.00000,-0.063420,0.011719,-5.4115,6.280584e-08,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
1393,z000,chr6_78,6,rs7453860,118565620.0,0.134310,T,C,0.98911,-0.061752,0.009660,-6.3928,1.644372e-10,0d6a9efa30af4548bb9021a1ae683260,0d6a9efa30af4548bb9021a1ae683260_z000,AO,static,0.645111,1.189926,0.391314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032811,z031,chr18_4,18,rs546321709,5307593.0,0.007045,T,C,0.52881,0.302600,0.056717,5.3352,9.587381e-08,d4a372ea29d8492592fae2971d274753,d4a372ea29d8492592fae2971d274753_z031,LV,dynamic,0.359540,0.781835,0.343003
1032896,z031,chr2_129,2,rs116059006,219217173.0,0.084510,C,T,0.99677,-0.061883,0.011611,-5.3298,9.876430e-08,d4a372ea29d8492592fae2971d274753,d4a372ea29d8492592fae2971d274753_z031,LV,dynamic,0.359540,0.781835,0.343003
1034332,z031,chr15_29,15,rs77310346,63814142.0,0.041617,C,T,0.99755,-0.095074,0.016402,-5.7965,6.817105e-09,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119
1034395,z031,chr16_41,16,16:77487414_TTG_T,77487414.0,0.367420,TTG,T,0.97306,0.037570,0.006922,5.4280,5.726642e-08,f18298663d2c43a39edec8e014ce66d9,f18298663d2c43a39edec8e014ce66d9_z031,RV,dynamic,0.758198,1.265266,0.457119


In [34]:
log10p_dynamic_df = -np.log10(results.loci_summary(only_dynamic=True).reset_index().groupby("region").min("min_P").sort_values("min_P")[["min_P"]])
log10p_static_df = -np.log10(results.loci_summary(only_static=True).reset_index().groupby("region").min("min_P").sort_values("min_P")[["min_P"]])

log10p_static_df.to_csv("results/log10p_static.csv")
log10p_dynamic_df.to_csv("results/log10p_dynamic.csv")

In [35]:
# "/home/rodrigo/01_repos/CardiacGWAS/manuscript/tables/gwas_counts_suggestive.tex"
results.create_count_table_tex("kk.tex")

Creating output file in kk.tex


In [39]:
results.region_assocs_df[results.region_assocs_df.P < 5e-10].\
  groupby(by=["run", "variable_type", "region"]).\
  aggregate({"CHR":"count", "P": "min"}).\
  rename({"CHR":"count", "P":"min_P"}, axis=1).\
  sort_values("count", ascending=False).\
  sort_values("min_P", ascending=True)

count         min_P
run                              variable_type region                       
f18298663d2c43a39edec8e014ce66d9 static        chr6_78       1  5.727960e-22
                                               chr6_79       1  1.584893e-21
45fb85430eb2419aba6953686d8e215b static        chr6_78       1  8.203515e-21
d4a372ea29d8492592fae2971d274753 static        chr6_78       1  1.741807e-20
                                 dynamic       chr6_78       5  2.328091e-20
...                                                        ...           ...
7f720206020d497ca329e89e9506e742 dynamic       chr12_19      1  4.254025e-10
0d6a9efa30af4548bb9021a1ae683260 dynamic       chr3_28       1  4.651575e-10
3223bc955cf54ac7bf7dd64356abbc76 dynamic       chr7_72       1  4.729334e-10
96aa5443bca34e5199114f8fcb1e5faf dynamic       chr20_22      1  4.870797e-10
db7aab61f9f94808b217437bb21e63b4 dynamic       chr10_46      1  4.968211e-10

[203 rows x 2 columns]

In [37]:
# region_count_df.sort_values("min_P").reset_index().to_csv("../00_CardiacMotionRL/analysis/loci_pvals_static_vs_dynamic.csv")

In [38]:
region_count_df = region_count_df.reset_index()
region_count_df = pd.merge(loci_mapping[["candidate_gene"]], region_count_df, left_index=True, right_on='region', how='right')
region_count_df[region_count_df.variable_type == "dynamic"].sort_values("min_P").head(30)

NameError: name 'region_count_df' is not defined

In [ ]:
COUNT_THR = 5
PVALUE_GW_THR = 5e-8
PVALUE_SW_THR = 5e-10

region_count_df[
    ((region_count_df["count"] >= COUNT_THR) & (region_count_df.min_P < PVALUE_GW_THR)) | (region_count_df.min_P < PVALUE_SW_THR)
].sort_values("min_P")

___

# Newer runs ($n_z^c=16$ and $n_z^s=16$)

In [ ]:
GWAS_RESULTS_DIR_NEWER = "/mnt/data/workshop/workshop-user1/output/GWAS/All_partitions_spatiotemporal_afterNov2023/"
results_newer = EnsembleResults(GWAS_RESULTS_DIR_NEWER)
results_newer.counts_per_chamber()

In [ ]:
COL_ORDER = [(variable_type, chamber) for variable_type in ["dynamic", "static"] for chamber in ["BV", "LV", "RV", "LA", "RA"]]

counts_per_chamber = results.loci_count(per_chamber=True).\
  reset_index().\
  pivot(index="region", values="count", columns=["variable_type", "chamber"]).\
  fillna(0).astype(int)[COL_ORDER]

___

In [ ]:
GWAS_RESULTS_DIR = "/mnt/data/workshop/workshop-user1/output/GWAS/relative_wall_thicknening/"
results = EnsembleResults(GWAS_RESULTS_DIR)

In [ ]:
results.region_assocs_df[(results.region_assocs_df.P < 5e-8)]

___

Export all SNPs

In [ ]:
import itertools

snps_list = []

significant_regions_df = region_count_df[
    ((region_count_df.min_P < 5e-8) & (region_count_df["count"] > 5)) | 
    (region_count_df.min_P < 1.5e-10)
]

for locus in significant_regions_df.index.to_list():
    snps_list.append(region_assocs_df.iloc[
        (region_assocs_df.index.get_level_values('region') == locus) 
        & (region_assocs_df.P < 5e-8).values,
    ])    
    # print(snps_list[-1])

with open("/home/rodrigo/SNPs.txt", "wt") as snps_file:
    snps_file.write(
      "\n".join(
        list(set([x for x in itertools.chain(*[x.SNP.values for x in snps_list])]))
      )
    )

In [ ]:
pd.concat(snps_list)[~pd.concat(snps_list).sort_values("SNP").duplicated("SNP")].\
    reset_index().\
    loc[:,["CHR", "BP", "region", "SNP"]].\
    to_csv("/home/rodrigo/01_repos/CardiacGWAS/significant_SNPs.csv")

This will filter the rows for the best associations per region:

In [ ]:
region_assocs_df.query("P < 1e-7"].to_csv("snp_associations_lt_1e-7.csv")

In [ ]:
kk = region_assocs_df.reset_index().set_index("SNP").loc[loci_mapping_df.lead_SNP.to_list()]
kk[~kk.duplicated("AF")]["AF"].to_csv("/home/rodrigo/Downloads/MAFs.txt")

In [ ]:
idx_min = region_assocs_df.groupby("region").P.idxmin()
idx_min = idx_min[significant_regions]

LEAD_SNPS_FILE = "/home/rodrigo/01_repos/CardiacMotionGWAS/results/snps_for_biomart__one_per_region.txt"
region_assocs_df.iloc[idx_min, [3,4,5,6]].reset_index(drop=True).sort_values(["CHR", "BP"]).to_csv(
    LEAD_SNPS_FILE, index=False
)

___

In [ ]:
regions_w = widgets.Select(options=sorted(results.get_significant_regions()))

@interact
def show_results_for_region(region=regions_w):
    display(results.get_results_for_region(region))

In [ ]:
by_region_and_snp_df = {}

for region in tqdm(region_assocs_df.index.get_level_values("region").unique()):
    
    df = region_assocs_df.iloc[
        (region_assocs_df.index.get_level_values('region') == region) 
        & (region_assocs_df.P < 5e-7).values,
    ].sort_values("P")
    
    if df.shape[0] < 5:
        continue
    
    # print(region)
    by_region_and_snp_df[region] = df

In [ ]:
@interact
def get_lead_snp_count(locus=regions_w):    
    
    return region_assocs_df.iloc[
        (region_assocs_df.index.get_level_values('region') == locus) 
        & (region_assocs_df.P < 5e-8).values,
    ].sort_values("P").head(20)

In [ ]:
### Replication results generated using this script: CardiacGWAS/replication.R

In [ ]:
with open("/home/rodrigo/01_repos/CardiacGWAS/manuscript/tables/replication_table.tex", "wt") as repl_f:
    dd = pd.read_csv("/home/rodrigo/01_repos/CardiacGWAS/results/replication_results.csv")
    dd['replication p-value'] = [f"${str(round(float(x[0]), 1))} \times 10^{{{int(x[1])}}}$" for x in dd['replication p-value'].apply(lambda x: f"{x:.2e}".split("e"))]
    table_code = dd.to_latex(escape=False, index=False)
    table_code = table_code.replace("_", "\_")
    repl_f.write(table_code)

___

Count loci

In [40]:
loci_mapping = fetch_loci_mapping()

NameError: name 'fetch_loci_mapping' is not defined

In [ ]:
condition = (loci_mapping.PCA != "YES") & (loci_mapping.exclude != "YES") & (loci_mapping.suggestive_significance != "YES") & (loci_mapping.handcrafted.isnull())

In [ ]:
loci_mapping.query("PCA != 'YES' and exclude != 'YES' and suggestive_significance != 'YES' and handcrafted.isnull()")